# Numerical approximation of the birth-immigration-death process

<!-- <center><font size="+4">Comparing the exact solution and diffusion approximation to the birth-immigration-death process</font></center> -->

## Birth-death

### Exact solution

In [1]:
apiece(tau, sigma) := -((sigma-1)*(%e^tau-1)) / 
    (sigma*%e^tau+%e^tau+sigma-1);
bpiece(tau, sigma) := (4*sigma^2*%e^tau) / 
    (sigma*%e^tau+%e^tau+sigma-1)^2;
cpiece(tau, sigma) := ((sigma+1)*(%e^tau-1)) /   
    (sigma*%e^tau+%e^tau+sigma-1);

                                                       tau
                                      - (sigma - 1) (%e    - 1)
(%o1)        apiece(tau, sigma) := -------------------------------
                                           tau     tau
                                   sigma %e    + %e    + sigma - 1

                                                  2   tau
                                           4 sigma  %e
(%o2)      bpiece(tau, sigma) := ----------------------------------
                                          tau     tau             2
                                 (sigma %e    + %e    + sigma - 1)

                                                      tau
                                       (sigma + 1) (%e    - 1)
(%o3)        cpiece(tau, sigma) := -------------------------------
                                           tau     tau
                                   sigma %e    + %e    + sigma - 1

In [2]:
probdistarr(tau, sigma, dist, nmax) := block(
    [a, b, c, d, k],
    a : apiece(tau,sigma),
    b : bpiece(tau,sigma),
    c : cpiece(tau, sigma),
    
    /* Generate the spike at n = 0. */
    dist[0] : a,
    /* Generate the rest of the distribution. */
    for k : 1 thru nmax do dist[k] : b * c^(k-1))$

In [3]:
convolve(a, b, c, nmax) := block(
    [m, k],
    for m : 0 thru nmax do(
        c[m] : 0,
        for k : 0 thru m do
            c[m] : c[m] + a[k] * b[m - k]))$

In [4]:
convolpow(a, m, b, nmax) := block(
    [tmparr, /* Temporary working array. */
    doubler /* Store consecutive doublings of a. */ ,
    n /* Counter for for loops. */ ],
    tmparr : make_array(flonum, nmax + 1),
    doubler : make_array(flonum, nmax + 1),
        
    /* Initialize the output array suitably. */
    if mod(m, 2) = 1 then(
        /* Set b = a when m is odd. */ 
        for n : 0 thru nmax do b[n] : a[n],
        m : m - 1)
        /* Set b to identity element when m is even. */
        else(
            b[0] : 1.0,
            for n : 1 thru nmax do b[n] : 0),
    m : m/2,
    
    /* Initialize the doubler. */
    for n : 0 thru nmax do doubler[n] : a[n],
    
    /*  Compute the result. */
    while m > 0 do(
        /* print("entering loop m= ", m), */
        /* Double the doubler. */
        for n : 0 thru nmax do tmparr[n] : doubler[n],
        convolve(tmparr, tmparr, doubler, nmax),
        /* When m is odd we multiply the doubler into b. */
        if mod(m, 2) = 1 then(
            for n : 0 thru nmax do tmparr[n] : b[n],
            convolve(tmparr, doubler, b, nmax),
            m : m - 1),
        m : m/2))$

In [5]:
exact_dist(tau, sigma, n0, nmax) := block(
    [dist0, distn],
    dist0 : make_array(flonum, nmax + 1),
    distn : make_array(flonum, nmax + 1),
    
    probdistarr(tau, sigma, dist0, nmax),
    convolpow(dist0, n0, distn, nmax),
    listarray(distn))$

### Diffusion approximation

In [6]:
diff_dist (xi, tau, xi0) :=
    (%e^(tau/2)*sqrt(xi0))/((%e^tau-1)*sqrt(xi)) *
    %e^(-(%e^tau*xi0+xi)/(%e^tau-1)) *
    bessel_i(1, 2*%e^(tau/2)*sqrt(xi*xi0)/(%e^tau-1));

                                                               tau
                                                          - (%e    xi0 + xi)
                                                          ------------------
                                     tau/2                      tau
                                   %e      sqrt(xi0)          %e    - 1
(%o8) diff_dist(xi, tau, xi0) := (--------------------) %e
                                     tau
                                  (%e    - 1) sqrt(xi)
                                                            tau/2
                                                        2 %e      sqrt(xi xi0)
                                            bessel_i(1, ----------------------)
                                                                tau
                                                              %e    - 1

### Comparing exact and diffusion

In [7]:
rescaled_exact(N, tau, s, x0, xmax) := block(
    [dist0, distn, nmax],
    nmax : N * xmax,
    dist0 : make_array(flonum, nmax + 1),
    distn : make_array(flonum, nmax + 1),
    
    probdistarr(tau, s/N, dist0, nmax),
    convolpow(dist0, N*x0, distn, nmax),
    makelist([n/N, N*distn[n]], n, 0, nmax))$

In [8]:
plot2d([[discrete, rescaled_exact(1, 1.0, 0.33, 5, 50)],
    [discrete, rescaled_exact(2, 1.0, 0.33, 5, 50)],
    [discrete, rescaled_exact(3, 1.0, 0.33, 5, 50)],
    [discrete, rescaled_exact(4, 1.0, 0.33, 5, 50)],
    0.66 * diff_dist(0.66*x, 1.0, 0.66*5.0)], [x, 0.1, 50.0],
    [legend, "N=1", "N=2", "N=3", "N=4", "diffusion"],
    [title, "tau = 1.0, s = 0.33, x_0 = 5"],
    [pdf_file, "fig/rescaled-exact.pdf"],
    [gnuplot_preamble, "set key right"],
    [gnuplot_pdf_term_command, "set term pdfcairo lw 3 size 17.2 cm, 12.9 cm font 'Latin Modern Roman,20'"]);

fig/rescaled-exact.pdf

(%o10)          [/tmp/maxout55.gnuplot, fig/rescaled-exact.pdf]

## Fix first two moments and vary sigma

Numerically calculate the cumulants.

In [9]:
cumus(dist) := block([len, m1, m2, m3],
    len : length(dist) - 1,
    m1 : sum(j * dist[j + 1], j, 0, len),
    m2 : sum(j^2 * dist[j + 1], j, 0, len),
    m3 : sum(j^3 * dist[j + 1], j, 0, len),
    [m1, m2 - m1^2, m3 - 3*m1*m2 + 2*m1^3])$

Analytically calculate cumulants of the BD process.

In [10]:
kaps(tau, n0, sigma) := [
    n0 * %e^(tau),
    (n0 / sigma) * (%e^(2*tau) - %e^(tau)),
    (n0 / (2 * sigma^2)) * (%e^(2*tau) - %e^(tau)) *
        (3*(%e^(tau) - 1) + (%e^(tau) + 1) * sigma^2)]$

Given values tau1 and n01, determine tau and n0 such that the BD distribution with parameters (tau, n0, sigma) has the same values of the first two moments as the BD distribution wiwth parameters (tau1, n01, 1).

In [11]:
solve((%e^tau-1)/sigma =%e^tau1-1, tau);

                                        tau1
(%o13)               [tau = log(sigma %e     - sigma + 1)]

In [12]:
psubst([tau=log(sigma*%e^tau1-sigma+1)], n01 * %e^(tau1 - tau));

                                        tau1
                                  n01 %e
(%o14)                     ------------------------
                                   tau1
                           sigma %e     - sigma + 1

In [13]:
momequiv (tau1, n01, sigma) :=
	    [log(sigma*%e^tau1-sigma+1), 
	    (n01*%e^tau1)/(sigma*%e^tau1-sigma+1)]$

Comparison for n01 = 12.

In [14]:
momequiv(0.1, 12, 0.2);
momequiv(0.1, 12, 0.8);

(%o16)             [0.02081601914619123, 12.98884134314821]

(%o17)             [0.08078403386384084, 12.23282137331797]

In [15]:
kaps(0.0208, 13, 0.2);
kaps(0.0808, 12, 0.8);

(%o18)    [13.27323175945365, 1.394872487412134, 0.5017851379449797]

(%o19)     [13.00984852888103, 1.368539208109951, 1.356838814088694]

In [16]:
plot2d([[discrete, exact_dist(0.0208, 0.2, 13, 30)], 
    [discrete, exact_dist(0.0808, 0.8, 12, 30)]],
    [legend, "tau = 0.2", "tau = 0.8"],
    [pdf_file, "fig/n01-12.pdf"],
    [gnuplot_preamble, "set key right"],
    [gnuplot_pdf_term_command, "set term pdfcairo lw 3 size 17.2 cm, 12.9 cm font 'Latin Modern Roman,20'"]);

fig/n01-12.pdf

(%o20)              [/tmp/maxout55.gnuplot, fig/n01-12.pdf]

In [17]:
cumus(exact_dist(0.0208, 0.2, 13, 30));
cumus(exact_dist(0.0808, 0.8, 12, 30));

(%o21)    [13.27323175945372, 1.394872487409998, 0.5017851379498097]

(%o22)     [13.00984852887727, 1.368539208090539, 1.356838813116156]

Comparison for n01 = 24

In [18]:
momequiv(0.1, 24, 0.2);
momequiv(0.1, 24, 0.8);

(%o23)             [0.02081601914619123, 25.97768268629641]

(%o24)             [0.08078403386384084, 24.46564274663594]

In [19]:
kaps(0.0208, 26, 0.2);
kaps(0.0808, 24, 0.8);

(%o25)     [26.5464635189073, 2.789744974824269, 1.003570275889959]

(%o26)     [26.01969705776206, 2.737078416219902, 2.713677628177388]

In [20]:
cumus(exact_dist(0.0208, 0.2, 26, 40));
cumus(exact_dist(0.0808, 0.8, 24, 40));

(%o27)     [26.5464634577302, 2.789745703590143, 1.003538327582646]

(%o28)     [26.01969669919773, 2.737082285673864, 2.713496631244197]

In [21]:
plot2d([[discrete, exact_dist(0.0208, 0.2, 26, 40)], 
    [discrete, exact_dist(0.0808, 0.8, 24, 40)]],
    [legend, "tau = 0.2", "tau = 0.8"],
    [pdf_file, "fig/n01-24.pdf"],
    [gnuplot_preamble, "set key right"],
    [gnuplot_pdf_term_command, "set term pdfcairo lw 3 size 17.2 cm, 12.9 cm font 'Latin Modern Roman,20'"]);

fig/n01-24.pdf

(%o29)              [/tmp/maxout55.gnuplot, fig/n01-24.pdf]

Comparison for n01 = 48

In [22]:
momequiv(0.1, 49, 0.2);
momequiv(0.1, 49, 0.8);

(%o30)             [0.02081601914619123, 53.03776881785518]

(%o31)              [0.08078403386384084, 49.9506872743817]

In [23]:
kaps(0.0208, 53, 0.2);
kaps(0.0808, 50, 0.8);

(%o32)     [54.11394486546487, 5.686787833295624, 2.045739408544917]

(%o33)     [54.20770220367096, 5.70224670045813, 5.653495058702892]

In [24]:
cumus(exact_dist(0.0208, 0.2, 53, 80));
cumus(exact_dist(0.0808, 0.8, 50, 80));

(%o34)     [54.11394486546723, 5.686787833164999, 2.045739415683784]

(%o35)     [54.20770220365917, 5.702246700775959, 5.65349503303878]

In [25]:
plot2d([[discrete, exact_dist(0.0208, 0.2, 53, 80)], 
    [discrete, exact_dist(0.0808, 0.8, 50, 80)]],
    [legend, "tau = 0.2", "tau = 0.8"],
    [pdf_file, "fig/n01-48.pdf"],
    [gnuplot_preamble, "set key right"],
    [gnuplot_pdf_term_command, "set term pdfcairo lw 3 size 17.2 cm, 12.9 cm font 'Latin Modern Roman,20'"]);

fig/n01-48.pdf

(%o36)              [/tmp/maxout55.gnuplot, fig/n01-48.pdf]

In [30]:
load(implicit_plot);

(%o41)     /usr/share/maxima/5.45.1/share/contrib/implicit_plot.lisp

In [ ]:
implicit_plot([(sigma^2/3) * coth(tau/2) = 0.25,
    (sigma^2/3) * coth(tau/2) = 0.5, 
    (sigma^2/3) * coth(tau/2) = 1.0,
    (sigma^2/3) * coth(tau/2) = 2.0,
    (sigma^2/3) * coth(tau/2) = 4.0,
    (sigma^2/3) * coth(tau/2) = 8.0], [sigma, 0.0, 1], [tau, 0.01, 0.5],
    [legend, false]
);


[(sigma^2/3) * coth(tau/2) = 0.25,
    (sigma^2/3) * coth(tau/2) = 0.5, 
    (sigma^2/3) * coth(tau/2) = 1.0,
    (sigma^2/3) * coth(tau/2) = 2.0,
    (sigma^2/3) * coth(tau/2) = 4.0,
    (sigma^2/3) * coth(tau/2) = 8.0], [sigma, 0.0, 1], [tau, 0.01, 0.5],
    [legend, false]

In [85]:
plot2d ([(sigma^2/3) * coth(tau/2) = 0.25,
         (sigma^2/3) * coth(tau/2) = 0.5, 
         (sigma^2/3) * coth(tau/2) = 1.0,
         (sigma^2/3) * coth(tau/2) = 2.0,
         (sigma^2/3) * coth(tau/2) = 4.0,
         (sigma^2/3) * coth(tau/2) = 8.0],
        [sigma, 0.0, 1], 
        [tau, 0.01, 0.5],
        [color, "#1F654C", "#226F54", "#559972", "#6EAE81", "#87C38F", "#BEDAA5"],
        [legend, false],
        [xlabel, "s"],
        [ylabel, "t"],
        [pdf_file, "fig/contour-plot-01.pdf"],
        [gnuplot_preamble, "
set label '0.25' at 0.3, 0.4 
set label '0.5' at 0.45, 0.37
set label '1.0' at 0.6, 0.3
set label '2.0' at 0.72, 0.21
set label '4.0' at 0.8, 0.14
set label '8.0' at 0.85, 0.08
set object circle at 0.2, 0.02 size 0.005 front fillstyle solid 1.0 fc rgb '#5B5B5B'
set label 'A' at 0.21, 0.02 front tc rgb '#5B5B5B'
set object circle at 0.8, 0.08 size 0.005 fillstyle solid 1.0 fc rgb '#5B5B5B'
set label 'B' at 0.81, 0.08 front tc rgb '#5B5B5B'
set object circle at 0.2, 0.2 size 0.005 fillstyle solid 1.0 fc rgb '#5B5B5B'
set label 'C' at 0.21, 0.2 front tc rgb '#5B5B5B'
set object circle at 0.8, 0.2 size 0.005 fillstyle solid 1.0 fc rgb '#5B5B5B'
set label 'D' at 0.81, 0.2 front tc rgb '#5B5B5B'"],
        [gnuplot_pdf_term_command, "set term pdfcairo enhanced lw 3.5 size 17.2 cm, 12.9 cm font 'Latin Modern Roman,23'"]);

fig/contour-plot-01.pdf

(%o84)         [/tmp/maxout55.gnuplot, fig/contour-plot-01.pdf]

## BID

Generate the distribution for the BD process with n_0 = 1 from the geometric series.  The answer goes into the array dist which runs from 0 to nmax.

In [26]:
BDdist(tau, sigma, dist, nmax) := block(
    [a, b, c, k],
    a : -((sigma-1)*(%e^tau-1)) / 
    (sigma*%e^tau+%e^tau+sigma-1),
    b : (4*sigma^2*%e^tau) / 
    (sigma*%e^tau+%e^tau+sigma-1)^2,
    c : ((sigma+1)*(%e^tau-1)) /   
    (sigma*%e^tau+%e^tau+sigma-1),
    
    /* Generate the spike at n = 0. */
    dist[0] : a,
    /* Generate the rest of the distribution. */
    for k : 1 thru nmax do dist[k] : b * c^(k-1))$

Given three arrays a, b, c which run form 0 to nmax, conolve a with b, writing the result into c. 

In [27]:
convolve(a, b, c, nmax) := block(
    [m, k],

    for m : 0 thru nmax do(
        c[m] : 0,
        for k : 0 thru m do
            c[m] : c[m] + a[k] * b[m - k]))$

Compute the repeated convolution of an array a with itself m times, writing the result into an array b.  Both arrays are assumed to run from 0 to nmax.

In [28]:
convolpow(a, m, b, nmax) := block(
    [tmparr, /* Temporary working array. */
    doubler /* Store consecutive doublings of a. */ ,
    n /* Counter for for loops. */ ],
    tmparr : make_array(flonum, nmax + 1),
    doubler : make_array(flonum, nmax + 1),
        
    /* Initialize the output array suitably. */
    if mod(m, 2) = 1 then(
        /* Set b = a when m is odd. */ 
        for n : 0 thru nmax do b[n] : a[n],
        m : m - 1)
        /* Set b to identity element when m is even. */
        else(
            b[0] : 1.0,
            for n : 1 thru nmax do b[n] : 0),
    m : m/2,
    
    /* Initialize the doubler. */
    for n : 0 thru nmax do doubler[n] : a[n],
    
    /*  Compute the result. */
    while m > 0 do(
        /* print("entering loop m= ", m), */
        /* Double the doubler. */
        for n : 0 thru nmax do tmparr[n] : doubler[n],
        convolve(tmparr, tmparr, doubler, nmax),
        /* When m is odd we multiply the doubler into b. */
        if mod(m, 2) = 1 then(
            for n : 0 thru nmax do tmparr[n] : b[n],
            convolve(tmparr, doubler, b, nmax),
            m : m - 1),
        m : m/2))$

Special case of the BID process with n_0 zero but m nonzero.

In [29]:
Idist(tau, sigma, m, dist, nmax) := block(
    [a, b, c, p, n],

    a : (1 + sigma - (1 - sigma) * %e^(-tau)) / 
         (2 * sigma * %e^(-tau)),
    b : (1 + sigma) * (1 - %e^(-tau)) /
         (1 + sigma - (1 - sigma) * %e^(-tau)),
    c : 2 * m * sigma / (1 + sigma),

    p : a^(-c),
    for n : 0 thru nmax do(
        dist[n] : p,
        p : p * b * (c + n) / (1 + n)))$